<a href="https://colab.research.google.com/github/vigilantmaster/RobotWithAvoidanceAndCommandCenter/blob/main/AutomataWithAvoidanceAndCommandCenter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This project uses a matrix to keep track of obstacles and the matrix gets updated when an object is detected.

# Web Server for Commands

In [ ]:
from flask import Flask, request, jsonify

app = Flask(__name__)

# Global variables for commands
command = {"target": None, "action": None}

@app.route('/send_command', methods=['POST'])
def send_command():
    data = request.json
    command["target"] = data.get("target")  # e.g., [3, 5]
    command["action"] = data.get("action")  # e.g., "honk"
    return jsonify({"status": "Command received", "command": command})

@app.route('/get_status', methods=['GET'])
def get_status():
    return jsonify({"location": robot.get_location(), "map": robot.get_map()})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)
#runs locally

#  Pathfinding with A*

In [ ]:
import heapq

class Robot:
    def __init__(self, grid_size):
        self.grid_size = grid_size
        self.map = [[0] * grid_size for _ in range(grid_size)]  # 0: Free, 1: Obstacle
        self.location = [0, 0]  # Starting location

    def update_map(self, obstacles):
        for x, y in obstacles:
            self.map[y][x] = 1

    def get_location(self):
        return self.location

    def get_map(self):
        return self.map

    def astar(self, start, goal):
        def heuristic(a, b):
            return abs(a[0] - b[0]) + abs(a[1] - b[1])

        open_set = []
        heapq.heappush(open_set, (0, start))
        came_from = {}
        g_score = {start: 0}
        f_score = {start: heuristic(start, goal)}

        while open_set:
            _, current = heapq.heappop(open_set)

            if current == goal:
                path = []
                while current in came_from:
                    path.append(current)
                    current = came_from[current]
                return path[::-1]

            for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                neighbor = (current[0] + dx, current[1] + dy)
                if 0 <= neighbor[0] < self.grid_size and 0 <= neighbor[1] < self.grid_size:
                    if self.map[neighbor[1]][neighbor[0]] == 1:  # Obstacle
                        continue

                    tentative_g_score = g_score.get(current, float('inf')) + 1
                    if tentative_g_score < g_score.get(neighbor, float('inf')):
                        came_from[neighbor] = current
                        g_score[neighbor] = tentative_g_score
                        f_score[neighbor] = tentative_g_score + heuristic(neighbor, goal)
                        heapq.heappush(open_set, (f_score[neighbor], neighbor))

        return None  # No path found

robot = Robot(grid_size=10) # very small space unless each location is a waypoint


# Navigation and Obstacle Avoidance

In [ ]:
import time

def navigate_to(target):
    while True:
        # Recalculate the path based on the current map
        path = robot.astar(tuple(robot.get_location()), tuple(target))
        if not path:
            print("No path to target!")
            return

        # Follow the path step by step
        for step in path:
            robot.location = list(step)  # Simulate movement
            print(f"Moving to {step}")
            time.sleep(0.5)  # Simulate delay

            # Simulate obstacle detection
            new_obstacles = detect_obstacles()  # Replace with sensor input
            if new_obstacles:
                robot.update_map(new_obstacles)
                print("New obstacles detected. Updating map...")
                break  # Recalculate path

        # Check if the robot has reached the target
        if tuple(robot.get_location()) == tuple(target):
            print(f"Reached target: {target}")
            return


def execute_action(action):
    print(f"Executing action: {action}")
    # Implement specific actions (e.g., honk, take a photo)


# Main Loop

In [ ]:
import threading

def main_loop():
    while True:
        if command["target"] and command["action"]:
            navigate_to(command["target"])
            execute_action(command["action"])
            command["target"], command["action"] = None, None

# Run the loop in a separate thread
threading.Thread(target=main_loop, daemon=True).start()


# Example Workflow
Open the web interface and send a command
The robot receives the command, plans a path to [5, 7], navigates there, and honks.

In [ ]:
{
    "target": [5, 7],
    "action": "honk"
}


# Things to Add

*   Use ultrasonic sensors or a camera for real-time obstacle detection.
*   Replace the static map with dynamic mapping (e.g., SLAM).
*   Visualize the grid and path in the web interface using JavaScript.






In [ ]:
import numpy as np

def calculate_object_position(robot_position, robot_orientation, distance, object_angle):
    """
    Calculate the object's position in the global coordinate system.

    robot_position: Tuple (x_robot, y_robot) - Robot's current grid location
    robot_orientation: Robot's orientation in radians (global frame)
    distance: Distance to the object (meters)
    object_angle: Angle to the object (radians) relative to the robot's forward direction
    """
    x_robot, y_robot = robot_position

    # Calculate the object's position in global coordinates
    global_angle = robot_orientation + object_angle
    x_object = x_robot + distance * np.cos(global_angle)
    y_object = y_robot + distance * np.sin(global_angle)

    return x_object, y_object


In [ ]:
def world_to_grid(x, y, grid_size, resolution):
    """
    Convert real-world coordinates to grid indices.

    x, y: Real-world coordinates (meters)
    grid_size: Tuple (width, height) of the grid in cells
    resolution: Size of each grid cell in meters
    """
    grid_x = int(x / resolution)
    grid_y = int(y / resolution)

    # Ensure the grid coordinates are within bounds
    grid_x = min(max(grid_x, 0), grid_size[0] - 1)
    grid_y = min(max(grid_y, 0), grid_size[1] - 1)

    return grid_x, grid_y


In [ ]:
def update_area_matrix(area_matrix, obstacle_position, grid_size, resolution):
    """
    Update the area matrix with the obstacle's position.

    area_matrix: 2D grid representing the environment
    obstacle_position: Tuple (x_object, y_object) in real-world coordinates
    grid_size: Tuple (width, height) of the grid
    resolution: Size of each grid cell in meters
    """
    grid_x, grid_y = world_to_grid(obstacle_position[0], obstacle_position[1], grid_size, resolution)
    area_matrix[grid_y][grid_x] = 1  # Mark as an obstacle
    return area_matrix


In [ ]:
# Robot's initial parameters
robot_position = (5.0, 5.0)  # Robot's position in meters
robot_orientation = np.radians(45)  # Robot's orientation in radians

# Detected object parameters
distance_to_object = 2.0  # Distance to object in meters
angle_to_object = np.radians(30)  # Angle to object in radians (relative to robot's forward direction)

# Environment grid
grid_size = (20, 20)  # Grid dimensions (20x20 cells)
resolution = 0.5  # Each cell is 0.5 meters
area_matrix = np.zeros(grid_size, dtype=int)  # Initialize grid

# Step 1: Calculate the object's global position
object_position = calculate_object_position(robot_position, robot_orientation, distance_to_object, angle_to_object)
print(f"Object position in real-world coordinates: {object_position}")

# Step 2: Update the area matrix
area_matrix = update_area_matrix(area_matrix, object_position, grid_size, resolution)

# Visualize the updated grid
print("Updated Area Matrix:")
print(area_matrix)


# Example Output
For the example above, with a robot at position (5, 5) and an object 2 meters away at a 30° angle:

Object's real-world position might be calculated as (6.732, 6.232).

Mapped grid cell: (13, 12) in a 20x20 grid with a resolution of 0.5 meters.
The area_matrix will have a 1 at the corresponding grid cell to indicate the obstacle.

